# Import Packages

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import os 
import matplotlib.pyplot as plt
import cv2 as cv
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Getting data from kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp /content/kaggle.json ~/.kaggle/


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d puneet6060/intel-image-classification

In [ ]:
! unzip /content/intel-image-classification.zip

In [ ]:
! rm /content/intel-image-classification.zip

## Data Preprocessing

In [ ]:
# checking image size
w = []
h = []

path = "/content/seg_pred/seg_pred"
for i in range(50):
  new_path = os.path.join(path, random.choice(os.listdir(path)))
  img = cv.imread(new_path)
  w.append(img.shape[0])
  h.append(img.shape[1])

sum(w)/len(w), sum(h)/len(w)

In [ ]:
# visualizing the image
plt.figure(figsize=(10, 10))
for i in range(1, 10):
  plt.subplot(4, 4, i)
  plt.tight_layout()
  img_path = os.path.join(path, random.choice(os.listdir(path)))
  img = plt.imread(img_path)
  plt.imshow(img)
plt.show()



In [ ]:
# creating data to feed model
# x = []
# y = []

# for i in os.listdir(path):
#   new_path = os.path.join(path, i)
#   for j in os.listdir(new_path):
#     y.append(i)
#     img_path = os.path.join(new_path, j)
#     img = plt.imread(img_path)
#     img = cv.resize(img, (150, 150))
#     x.append(img)

# del img
# x = np.array(x)
# y = np.array(y)

In [ ]:
path = "/content/seg_train/seg_train"

In [ ]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,validation_split=0.2)

In [ ]:
train_dataset = image_generator.flow_from_directory(batch_size=128,
                                                 directory=path,
                                                 shuffle=True,
                                                 target_size=(150, 150), 
                                                 subset="training",
                                                 class_mode='categorical')

validation_dataset = image_generator.flow_from_directory(batch_size=128,
                                                 directory=path,
                                                 shuffle=True,
                                                 target_size=(150, 150), 
                                                 subset="validation",
                                                 class_mode='categorical')

In [ ]:
train_dataset.image_shape, train_dataset.class_indices

# Building Model

In [ ]:
model = keras.models.Sequential()

In [ ]:
# adding layers
model.add(keras.layers.Conv2D(filters=8, kernel_size=(2, 2), activation='relu', input_shape=(150, 150, 3), padding='same'))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Conv2D(filters=32, kernel_size=(4, 4), activation="relu", padding='same'))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding='same'))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Conv2D(filters=128, kernel_size=(2, 2), activation="relu", padding='same'))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.2))


model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(units=32, activation='relu'))
model.add(keras.layers.Dense(units=6, activation='softmax'))

In [ ]:
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# training the model
history = model.fit(train_dataset, epochs=15, validation_data=validation_dataset)

In [ ]:
# evaluation model

print(model.evaluate(train_dataset))

In [ ]:
print(model.evaluate(validation_dataset))

In [ ]:
# creating model
path = '/content/seg_pred/seg_pred'
x_test = []

for i in os.listdir(path):
  img_path = os.path.join(path, i)
  img = plt.imread(img_path)
  img = cv.resize(img, (150, 150))

  x_test.append(img)

In [ ]:
x_test = np.array(x_test)
x_test = x_test / 255

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(1, 10):
  plt.subplot(4, 4, i)
  plt.tight_layout()
  plt.imshow(x_test[i])
  plt.title(np.argmax(model.predict(x_test[i].reshape(1, 150, 150, 3))))
plt.show()

In [ ]:
train_dataset.class_indices